In [1]:
words= open('names_indian.txt', 'r').read().splitlines()

In [2]:
words[:10]

['Aachal',
 'Aadharsh',
 'Aadhavi',
 'Aadhira',
 'Aadidev',
 'Aadil',
 'Aadita',
 'Aaditya',
 'Aadiv',
 'Aadrik']

In [3]:
len(words)

2195

In [4]:
min(len(w) for w in words)

2

In [5]:
max(len(w) for w in words)

14

In [6]:
words_new = [word.lower() for word in words]

In [7]:
tri={}
for w in words_new[:3]:
     chs=['<S>']+list(w)+['<E>'] #adding a starting and ending character
     for ch1,ch2,ch3 in zip(chs,chs[1:],chs[2:]):
         trigram=(ch1,ch2,ch3)
         tri[trigram]=tri.get(trigram,0)+1

In [8]:
tri

{('<S>', 'a', 'a'): 3,
 ('a', 'a', 'c'): 1,
 ('a', 'c', 'h'): 1,
 ('c', 'h', 'a'): 1,
 ('h', 'a', 'l'): 1,
 ('a', 'l', '<E>'): 1,
 ('a', 'a', 'd'): 2,
 ('a', 'd', 'h'): 2,
 ('d', 'h', 'a'): 2,
 ('h', 'a', 'r'): 1,
 ('a', 'r', 's'): 1,
 ('r', 's', 'h'): 1,
 ('s', 'h', '<E>'): 1,
 ('h', 'a', 'v'): 1,
 ('a', 'v', 'i'): 1,
 ('v', 'i', '<E>'): 1}

In [9]:
sorted(tri.items(), key= lambda kv:-kv[1])

[(('<S>', 'a', 'a'), 3),
 (('a', 'a', 'd'), 2),
 (('a', 'd', 'h'), 2),
 (('d', 'h', 'a'), 2),
 (('a', 'a', 'c'), 1),
 (('a', 'c', 'h'), 1),
 (('c', 'h', 'a'), 1),
 (('h', 'a', 'l'), 1),
 (('a', 'l', '<E>'), 1),
 (('h', 'a', 'r'), 1),
 (('a', 'r', 's'), 1),
 (('r', 's', 'h'), 1),
 (('s', 'h', '<E>'), 1),
 (('h', 'a', 'v'), 1),
 (('a', 'v', 'i'), 1),
 (('v', 'i', '<E>'), 1)]

In [10]:
import torch
import torch.nn.functional as F

In [11]:
chars=sorted(list(set(''.join(words_new))))
stoi ={s:i+1 for i,s in enumerate(chars)}
stoi['.']=0
itos={i:s for s,i in stoi.items()}

In [12]:
stoi

{'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26,
 '.': 0}

The Neural Network

In [25]:
#creating the dataset intially with just 3 words to experiment
xs=[]
ys=[]
for w in words_new[:3]: 
     chs=['.']+list(w)+['.'] 
     for ch1,ch2,ch3 in zip(chs,chs[1:],chs[2:]):
        ix1=stoi[ch1]
        ix2=stoi[ch2]
        ix3=stoi[ch3] 
        xs.append([ix1,ix2]) 
        ys.append(ix3)
xs=torch.tensor(xs)        
ys=torch.tensor(ys) 

In [26]:
xs

tensor([[ 0,  1],
        [ 1,  1],
        [ 1,  3],
        [ 3,  8],
        [ 8,  1],
        [ 1, 12],
        [ 0,  1],
        [ 1,  1],
        [ 1,  4],
        [ 4,  8],
        [ 8,  1],
        [ 1, 18],
        [18, 19],
        [19,  8],
        [ 0,  1],
        [ 1,  1],
        [ 1,  4],
        [ 4,  8],
        [ 8,  1],
        [ 1, 22],
        [22,  9]])

In [27]:
ys

tensor([ 1,  3,  8,  1, 12,  0,  1,  4,  8,  1, 18, 19,  8,  0,  1,  4,  8,  1,
        22,  9,  0])

In [28]:
xs.shape

torch.Size([21, 2])

In [29]:
ys.shape

torch.Size([21])

The Data

In [30]:
#DataSet Creation
xs=[]
ys=[]
for w in words_new: 
     chs=['.']+list(w)+['.'] 
     for ch1,ch2,ch3 in zip(chs,chs[1:],chs[2:]):
        ix1=stoi[ch1]
        ix2=stoi[ch2]
        ix3=stoi[ch3] 
        xs.append([ix1,ix2]) 
        ys.append(ix3)
xs=torch.tensor(xs)        
ys=torch.tensor(ys) 
num = xs.shape[0]
num


14187

The Model

In [31]:
g = torch.Generator().manual_seed(2147555555)
w=torch.rand((54,27),generator=g,requires_grad=True) # 27 neurons

In [32]:
# gradient descent
for k in range(50):
    #Forward Pass starts here
    xenc = F.one_hot(xs, num_classes=27).float() # xenc shape would be [21,2,27]
    xenc_flat = xenc.view(xs.shape[0], -1) #flattent the xenc shape to [21,54]
    logits=xenc_flat @ w
    counts = logits.exp() # counts, equivalent to N
    probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
    loss = -probs[torch.arange(num), ys].log().mean()
    print(loss.item())

    # backward pass starts here
    w.grad = None # set to zero the gradient
    loss.backward()
      
    # update
    w.data += -20 * w.grad

3.3947997093200684
3.058293342590332
2.823643207550049
2.6804776191711426
2.5917601585388184
2.5299558639526367
2.482943296432495
2.4452905654907227
2.4141435623168945
2.387815475463867
2.365213394165039
2.3455746173858643
2.328336000442505
2.3130691051483154
2.29944109916687
2.2871899604797363
2.276106595993042
2.266023874282837
2.2568047046661377
2.2483365535736084
2.2405269145965576
2.233297824859619
2.226583242416382
2.2203269004821777
2.2144813537597656
2.2090046405792236
2.2038614749908447
2.1990206241607666
2.1944544315338135
2.1901392936706543
2.18605375289917
2.1821794509887695
2.178499221801758
2.1749982833862305
2.171663284301758
2.1684823036193848
2.1654441356658936
2.162539005279541
2.1597578525543213
2.1570932865142822
2.1545369625091553
2.152082681655884
2.149724006652832
2.1474552154541016
2.145271062850952
2.143167018890381
2.1411380767822266
2.1391799449920654
2.1372900009155273
2.135463237762451


In [33]:
g = torch.Generator().manual_seed(2147555555)
for _ in range(10):  # Generate 10 words
    out = []
    context = [stoi['.'], stoi['.']]  # Starting context
    while True:
        x = torch.tensor([context])
        xenc = F.one_hot(x, num_classes=27).float()
        xenc_flat = xenc.view(1, -1)  # [1, 54]
        logits = xenc_flat @ w  # [1, 27]
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1).item()
        if ix == stoi['.']:
            break
        out.append(itos[ix])
        context = [context[1], ix]  # Slide window
    print(''.join(out))


ueshdfa
keshan
an
ruj
uatinishvka
uan
urth
ana
ruthiacjan
vagnd
